In [19]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [20]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

In [21]:
model_checkpoint = "t5-small"

In [22]:
from datasets import load_dataset
raw_datasets  = load_dataset("opus_books", "en-fr")

In [23]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [25]:
raw_datasets['train'][10]

{'id': '10',
 'translation': {'en': 'My mother taught the infants.',
  'fr': 'Ma mère faisait la petite classe.'}}

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [27]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [29]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["Ma mère faisait la petite classe."]))

{'input_ids': [[1534, 18052, 22669, 17, 50, 6882, 853, 15, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [30]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "fr"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [31]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[37, 12832, 49, 1], [901, 9, 77, 18, 371, 1211, 8632, 1]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[312, 1907, 1212, 9, 83, 1496, 1], [901, 9, 77, 18, 371, 1211, 8632, 1]]}

In [32]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [33]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [34]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [36]:
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)

In [42]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25417
    })
})

In [44]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

test_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [45]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [46]:
model.fit(train_dataset, validation_data=test_dataset, epochs=5)

Epoch 1/5
1588/1588 [==============================] - 629s 381ms/step - loss: 2.2008 - val_loss: 1.7666
Epoch 2/5
1588/1588 [==============================] - 598s 376ms/step - loss: 1.9918 - val_loss: 1.6903
Epoch 3/5
1588/1588 [==============================] - 601s 379ms/step - loss: 1.9230 - val_loss: 1.6455
Epoch 4/5
1588/1588 [==============================] - 595s 375ms/step - loss: 1.8756 - val_loss: 1.6091
Epoch 5/5
1588/1588 [==============================] - 635s 400ms/step - loss: 1.8434 - val_loss: 1.5813


In [47]:
model.save_pretrained("tf_model/")

In [48]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [49]:
input_text  = "My name Rahul. Just Testing the language translation"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[    0  2963  5252 20492    83     6     3   173 10286    17    20 18924
     50 16486  1575     3   697    76   343  1495     5     1]], shape=(1, 22), dtype=int32)


In [50]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

Mon nom Rahul, il suffit de tester la traduction linguistique.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
